In [1]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Activation, Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import layers, models



import pandas as pd
import numpy as np

#from keras.preprocessing import image
import keras.utils as image
from matplotlib import pyplot as plt

import cv2

import glob
import os

In [ ]:
import matplotlib.colors
import imageio
import scipy, scipy.misc, scipy.signal
import cv2
import sys

def build_is_hist(img):
    hei = img.shape[0]
    wid = img.shape[1]
    ch = img.shape[2]
    Img = np.zeros((hei+4, wid+4, ch))
    for i in range(ch):
        Img[:,:,i] = np.pad(img[:,:,i], (2,2), 'edge')
    hsv = (matplotlib.colors.rgb_to_hsv(Img))
    hsv[:,:,0] = hsv[:,:,0] * 255
    hsv[:,:,1] = hsv[:,:,1] * 255
    hsv[hsv>255] = 255
    hsv[hsv<0] = 0
    hsv = hsv.astype(np.uint8).astype(np.float64)
    fh = np.array([[-1.0,0.0,1.0],[-2.0,0.0,2.0],[-1.0,0.0,1.0]])
    fv = fh.conj().T
    
    H = hsv[:,:,0]
    S = hsv[:,:,1]
    I = hsv[:,:,2]

    dIh = scipy.signal.convolve2d(I, np.rot90(fh, 2), mode='same')
    dIv = scipy.signal.convolve2d(I, np.rot90(fv, 2), mode='same')
    dIh[dIh==0] = 0.00001
    dIv[dIv==0] = 0.00001
    dI = np.sqrt(dIh**2+dIv**2).astype(np.uint32)
    di = dI[2:hei+2,2:wid+2]
    
    dSh = scipy.signal.convolve2d(S, np.rot90(fh, 2), mode='same')
    dSv = scipy.signal.convolve2d(S, np.rot90(fv, 2), mode='same')
    dSh[dSh==0] = 0.00001
    dSv[dSv==0] = 0.00001
    dS = np.sqrt(dSh**2+dSv**2).astype(np.uint32)
    ds = dS[2:hei+2,2:wid+2]

    
    h = H[2:hei+2,2:wid+2]
    s = S[2:hei+2,2:wid+2]
    i = I[2:hei+2,2:wid+2].astype(np.uint8)
    
    Imean = scipy.signal.convolve2d(I,np.ones((5,5))/25, mode='same')
    Smean = scipy.signal.convolve2d(S,np.ones((5,5))/25, mode='same')
    
    Rho = np.zeros((hei+4,wid+4))
    for p in range(2,hei+2):
        for q in range(2,wid+2):
            tmpi = I[p-2:p+3,q-2:q+3]
            tmps = S[p-2:p+3,q-2:q+3]
            corre = np.corrcoef(tmpi.flatten('F'),tmps.flatten('F'))
            Rho[p,q] = corre[0,1]
    
    rho = np.abs(Rho[2:hei+2,2:wid+2])
    rho[np.isnan(rho)] = 0
    rd = (rho*ds).astype(np.uint32)
    Hist_I = np.zeros((256,1))
    Hist_S = np.zeros((256,1))
    
    for n in range(0,255):
        temp = np.zeros(di.shape)
        temp[i==n] = di[i==n]
        Hist_I[n+1] = np.sum(temp.flatten('F'))
        temp = np.zeros(di.shape)
        temp[i==n] = rd[i==n]
        Hist_S[n+1] = np.sum(temp.flatten('F'))

    return Hist_I, Hist_S

def dhe(img, alpha=0.5):
    
    hist_i, hist_s = build_is_hist(img)
    hist_c = alpha*hist_s + (1-alpha)*hist_i
    hist_sum = np.sum(hist_c)
    hist_cum = hist_c.cumsum(axis=0)
    
    hsv = matplotlib.colors.rgb_to_hsv(img)
    h = hsv[:,:,0]
    s = hsv[:,:,1]
    i = hsv[:,:,2].astype(np.uint8)
    
    c = hist_cum / hist_sum
    s_r = (c * 255)
    i_s = np.zeros(i.shape)
    for n in range(0,255):
        i_s[i==n] = s_r[n+1]/255.0
    i_s[i==255] = 1
    hsi_o = np.stack((h,s,i_s), axis=2)
    result = matplotlib.colors.hsv_to_rgb(hsi_o)
    
    result = result * 255
    result[result>255] = 255
    result[result<0] = 0
    return result.astype(np.uint8)

In [ ]:
SIZE= 224

train_images = []
train_labels = []
train_dhe=[]
for directory_path in glob.glob("brain-tumor\\Training\\*"):
#for directory_path in "/content/drive/My Drive/dataset/images/validation/*":
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
    #for img_path in directory_path, "*.jpg":
        #print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        #try:
        img = cv2.resize(img, (SIZE,SIZE))



        img = image.img_to_array(img)
        ## CLAHE dosu
        img = img.astype(np.uint8)
        #lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l,a,b = cv2.split(lab_img)
        equ = cv2.equalizeHist(l)
        updated_lab_img1 = cv2.merge((equ,a,b))
        hist_eq_img = cv2.cvtColor(updated_lab_img1,cv2.COLOR_LAB2BGR)
    
        ##Apply CLAHE to Lchannel
        clahe = cv2.createCLAHE(clipLimit=3.0,tileGridSize=(8,8))
        clahe_img = clahe.apply(l)
    
        ## combine the CLAHE
        updated_lab_img2 =cv2.merge((clahe_img,a,b))
    
        ## convert LAB
        CLAHE_img = cv2.cvtColor(updated_lab_img2, cv2.COLOR_LAB2BGR)
        #####result = dhe(img) 
    
        #except:
        #break
        #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        ####train_dhe.append(result)
        train_images.append(CLAHE_img)
        train_labels.append(label)

###train_dhe = np.array(train_dhe)
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
SIZE= 224

test_images = []
test_labels = []
for directory_path in glob.glob("brain-tumor\\Testing\\*"):
#for directory_path in "/content/drive/My Drive/dataset/images/validation/*":
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
    #for img_path in directory_path, "*.jpg":
        #print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        #try:
        img = cv2.resize(img, (SIZE,SIZE))



        img = image.img_to_array(img)
        ## CLAHE dosu
        img = img.astype(np.uint8)
        #lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l,a,b = cv2.split(lab_img)
        equ = cv2.equalizeHist(l)
        updated_lab_img1 = cv2.merge((equ,a,b))
        hist_eq_img = cv2.cvtColor(updated_lab_img1,cv2.COLOR_LAB2BGR)
    
        ##Apply CLAHE to Lchannel
        clahe = cv2.createCLAHE(clipLimit=3.0,tileGridSize=(8,8))
        clahe_img = clahe.apply(l)
    
        ## combine the CLAHE
        updated_lab_img2 =cv2.merge((clahe_img,a,b))
    
        ## convert LAB
        CLAHE_img = cv2.cvtColor(updated_lab_img2, cv2.COLOR_LAB2BGR)

    
        #except:
        #break
        #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(CLAHE_img)
        test_labels.append(label)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
#Encode Labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)

In [ ]:
y_train = train_labels_encoded
yv_test = test_labels_encoded

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x2_test, y_train,  y_test = train_test_split(train_images, train_labels_encoded, random_state=20, test_size=0.3)

In [ ]:
#Normalise pixel values to between 0 and 1
x_train, x2_test, xv_test = x_train /255.0, x2_test / 255.0, test_images/255.0

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

yv_test_one_hot = to_categorical(yv_test)

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(SIZE,SIZE,3))

In [ ]:
prep_resnettr = preprocess_input(x_train)
prep_resnette = preprocess_input(x2_test)
prep_v = preprocess_input(xv_test)

resnet_trfeature = resnet_model.predict(prep_resnettr)
resnet_tefeature = resnet_model.predict(prep_resnette)
resnet_v = resnet_model.predict(prep_v)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models


def cbam_block(inputs, reduction_ratio=0.5):
    # Channel attention
    channels = inputs.shape[-1]
    avg_pool = layers.GlobalAveragePooling2D()(inputs)
    max_pool = layers.GlobalMaxPooling2D()(inputs)
    shared_layer_1 = layers.Dense(units=int(channels * reduction_ratio), activation='relu', use_bias=True)
    shared_layer_2 = layers.Dense(units=channels, activation='relu', use_bias=True)
    avg_pool = shared_layer_1(avg_pool)
    avg_pool = shared_layer_2(avg_pool)
    max_pool = shared_layer_1(max_pool)
    max_pool = shared_layer_2(max_pool)
    attention = layers.Add()([avg_pool, max_pool])
    attention = layers.Activation('sigmoid')(attention)
    attention = layers.Reshape((1, 1, channels))(attention)
    scaled_inputs = layers.Multiply()([inputs, attention])

    # Spatial attention
    squeeze = layers.Conv2D(filters=1, kernel_size=1, activation='sigmoid', use_bias=False)(scaled_inputs)
    expanded_inputs = layers.Multiply()([scaled_inputs, squeeze])

    return expanded_inputs


def ResNet50_CBAM():
    input_shape = (224, 224, 3)

    # Create the ResNet50 model
    resnet50 = tf.keras.applications.ResNet50(include_top=False, input_shape=input_shape)

    # Add the CBAM attention mechanism
    x = resnet50.output
    x = cbam_block(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(units=4, activation='sigmoid')(x)

    # Combine the ResNet50 model and the attention mechanism
    model = models.Model(inputs=resnet50.input, outputs=x, name='ResNet50_CBAM')

    return model

In [ ]:
# Define the ResNet50 model with added CBAM
def ResNet50_CBAM():
    input_shape = (224, 224, 3)
    
    # Create the ResNet50 model
    resnet50 = tf.keras.applications.ResNet50(include_top=False, input_shape=input_shape)
    
    # Add the CBAM attention mechanism
    x = layers.GlobalAveragePooling2D()(resnet50.output)
    x = layers.Dense(units=1024, activation='relu')(x)
    #x = layers.Dropout(0.5)(x)
    x = layers.Dense(units=512, activation='relu')(x)
    #x = layers.Dropout(0.5)(x)
    x = layers.Dense(units=256, activation='relu')(x)
    #x = layers.Dropout(0.5)(x)
    x = layers.Dense(units=128, activation='relu')(x)
    print(x.shape)
    x = layers.Dropout(0.5)(x)
    
    
    x = layers.Reshape((1, 1, 128))(x)
    x_max = layers.GlobalMaxPooling2D()(x)
    x_avg = layers.GlobalAveragePooling2D()(x)
    x = layers.Concatenate()([x_max, x_avg])
    x = layers.Dense(units=4,activation='sigmoid')(x)
    
    # Combine the ResNet50 model and the attention mechanism
    model = models.Model(inputs=resnet50.input, outputs=x, name='ResNet50_CBAM')
    
    return model

In [ ]:
model = ResNet50_CBAM()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

# train the model
history = model.fit(train_dhe, train_labels_encoded, epochs=20, batch_size=32, validation_data=(test_dhe, test_labels_encoded))